# Modular CerberusTS Experimentation

First we make sure we are in the base working directory (this is probably not necessary if using the package):

In [1]:
import os
os.chdir("..")

Then we import the necessary CerberusTS modules. 

In [2]:
from cerberus_ts import Cerberus, train_cerberus
from cerberus_ts import TimeseriesDataPreparer, ResponseGenerator
from cerberus_ts import Foresight, train_foresight

import pandas as pd

c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here we have the ability to do some custom configuration:

In [3]:
from cerberus_ts import CerberusConfig
CerberusConfig.set_masked_norm_zero = True

## Dataset Loading

Next we need a Pandas Dataframe of some wide-formatted mutlivariate timeseries data. Here we will use the Jena Climate Data found here: https://www.kaggle.com/datasets/mnassrib/jena-climate

In [4]:
df = pd.read_csv(r"data/jena_climate_2009_2016.csv",
                parse_dates=['Date Time'], 
                index_col=['Date Time'])
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
df = df.iloc[:5000,:]

## User Argument Specification
There are a number of different arguments that can be provided to Cerberus, shown below:

In [5]:
# What ratio of data needs to be available for the various call, response, and context windows to use that timestamp?
thresholds = {
    'call': 1,
    'response': 1,
    'context_0': 1,
    'context_1': 1,
    'context_2': 1
}

# How big should the call, context(s), and response windows be?
sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

# What timestep should each head use?
window_timesteps = {
    'call': '10T',
    'response': '10T',
    'context_0': '1H',
    'context_1': '2H',
    'context_2': '6H'
}

# Which columns should be used for each head?
feature_indexes = {
    'call': range(0,14),
    'response': [0, 1, 4],
    'context_0': range(0,14),
    'context_1': range(0,14),
    'context_2': range(0,14)   
}

## Prepare Dataset

We will use CerberusTS's TimeseriesDataPreparer class to create Torch dataloaders for all our different heads, scaled, downsampled, and coil-normalized for easy Cerberus use. 

In [6]:
# Initialize the preparer
preparer = TimeseriesDataPreparer(df, sizes, thresholds, feature_indexes, window_timesteps, train_len = 20_000, feature_range = (0, 1), batch_size = 100)

# Prepare the data
preparer.prepare_data()

## Foresight Training

First, we can optionally train Foresight to aid CerberusTS:

In [7]:
foresight = Foresight(sizes,feature_indexes, csize = 128, head_layers=["mha"])

In [8]:
foresight = train_foresight(foresight, preparer.dataloaders, num_epochs = 30)

using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA
using MHA


Traceback (most recent call last):
  File "C:\Users\johnm\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\johnm\AppData\Local\Temp\ipykernel_17920\1705153153.py", line 1, in <module>
    foresight = train_foresight(foresight, preparer.dataloaders, num_epochs = 30)
  File "c:\Users\johnm\Documents\GitHub\cerberusPy\cerberus_ts\modules\build_cerberus.py", line 326, in train_foresight
    loss = criterion(outputs, unmasked_batch)
  File "c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "c:\Users\johnm\Documents\GitHub\cerberusPy\cerberus_ts\modules\build_cerb

## CerberusTS Training

With the Foresight model trained (and weights frozen), we can pass this into a Cerberus model and train the remaining weights to generate predictions. 

In [ ]:
model = Cerberus(sizes, feature_indexes, foresight=foresight, head_layers=["conv","conv"])

In [ ]:
model = train_cerberus(model, preparer.dataloaders, num_epochs = 60)

## Results Review

CerberusTS has some built in functionality for generating responses as well as visualizing the results. 

### Normalized Response Review

In [ ]:
# Intialize Response Generator
generator = ResponseGenerator(model, preparer.sliced_data, feature_indexes, preparer.max_change_dfs)

# Generate a response for a specific index
sel_index = 2130
generator.generate_response(sel_index)

generator.plot_normalized_responses()

### Unscaled Response Review

In [ ]:
generator.plot_unscaled_responses(preparer.min_max_df, feature_indexes)